In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!free -g

              total        used        free      shared  buff/cache   available
Mem:             12           1           2           0           9          11
Swap:             0           0           0


In [6]:
!pip3 install imageio

    100% |████████████████████████████████| 3.3MB 8.7MB/s 


In [0]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.models import load_model

import matplotlib.pyplot as plt

import sys
import os
import numpy as np
import imageio

In [0]:
img_rows = 56
img_cols = 56
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [0]:
def build_discriminator():
  
  model = Sequential()

  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(56,56,3), padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(ZeroPadding2D(padding=((0,1),(0,1))))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  img = Input(shape=img_shape)
  validity = model(img)

  return Model(img, validity)

In [0]:
def build_generator():

  model = Sequential()

  model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
  model.add(Reshape((7, 7, 128)))
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  model.add(Conv2D(64, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  #         

  model.add(UpSampling2D())
  model.add(Conv2D(32, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  #         
  model.add(Conv2D(channels, kernel_size=3, padding="same"))
  model.add(Activation("tanh"))

  #         model.summary()

  noise = Input(shape=(latent_dim,))
  img = model(noise)

  return Model(noise, img)


In [0]:
def model_train(epochs,discriminator,generator, combined_model, batch_size=64, save_interval=10):

  # Load the dataset
  path = 'drive/projects/SpriteGAN/pokemon_sprites'

  nb_samples = len(os.listdir(path))
  #         print(nb_samples)

  X_train = np.zeros(shape=(nb_samples, img_rows, img_cols,3), dtype= 'float32')

  for i,f in enumerate(os.listdir(path)):
    img_path = os.path.join(path, f)
    img = imageio.imread(img_path)
    img_no_alpha = img[:,:,:3]
    if img_no_alpha.shape != X_train[0].shape:
      npad = ((int((img_rows-img_no_alpha.shape[0])/2), int((img_rows-img_no_alpha.shape[0])/2)), \
              (int((img_cols-img_no_alpha.shape[1])/2), int((img_cols-img_no_alpha.shape[1])/2)), (0, 0))
      padded_img = np.pad(img_no_alpha, pad_width=npad, mode='constant', constant_values=255)
      X_train[i] = padded_img
      continue
    X_train[i] = img_no_alpha          



  # Rescale -1 to 1
  X_train = X_train / 255.
  #         X_train = np.expand_dims(X_train, axis=3)

  # Adversarial ground truths
  valid = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for epoch in range(epochs):

      # ---------------------
      #  Train Discriminator
      # ---------------------
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]

      # Sample noise and generate a batch of new images
      noise = np.random.normal(-1, 1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      # Train the discriminator (real classified as ones and generated as zeros)
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      # ---------------------
      #  Train Generator
      # ---------------------

      # Train the generator (wants discriminator to mistake images as real)
      g_loss = combined_model.train_on_batch(noise, valid)

      # Plot the progress
      print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

      # If at save interval => save generated image samples
      if epoch % save_interval == 0:
          save_imgs(epoch,generator)

In [0]:
def save_imgs(epoch,generator):
  r, c = 1, 5
  noise = np.random.normal(-1, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)
  plt.imshow(np.clip(gen_imgs[0],0,1))

  plt.savefig("drive/projects/SpriteGAN/Images/spriteGAN2_%d.png" % epoch)
  plt.close()

In [0]:
optimizer = Adam(0.0002, 0.5)
# Build and compile the discriminator
path = "drive/projects/SpriteGAN/"
if os.path.exists(os.path.join(path,'discriminator.h5')):
  print("Loading previous discriminator..")
  discriminator = load_model(os.path.join(path,'discriminator.h5'))
else:
  discriminator = build_discriminator()
  
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the generator
if os.path.exists(os.path.join(path,'generator.h5')):
  print("Loading previous generator..")
  generator = load_model(os.path.join(path,'generator.h5'))
else:
  generator = build_generator()


# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
if os.path.exists(os.path.join(path,'combined_model.h5')):
  print("Loading previous combined model..")
  combined_model = load_model(os.path.join(path,'combined_model.h5'))
else:
  combined_model = Model(z, valid)
  
combined_model.compile(loss='binary_crossentropy', optimizer=optimizer)

In [52]:
model_train(5001,discriminator,generator, combined_model,save_interval=500)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.294358, acc.: 33.59%] [G loss: 0.636564]
1 [D loss: 1.003131, acc.: 54.69%] [G loss: 1.165860]
2 [D loss: 0.741150, acc.: 57.03%] [G loss: 1.933183]
3 [D loss: 0.576534, acc.: 68.75%] [G loss: 2.207663]
4 [D loss: 0.433298, acc.: 79.69%] [G loss: 2.527285]
5 [D loss: 0.285690, acc.: 90.62%] [G loss: 1.757740]
6 [D loss: 0.185228, acc.: 96.09%] [G loss: 1.084995]
7 [D loss: 0.127804, acc.: 98.44%] [G loss: 0.910445]
8 [D loss: 0.135969, acc.: 98.44%] [G loss: 0.785740]
9 [D loss: 0.089927, acc.: 99.22%] [G loss: 0.886673]
10 [D loss: 0.113989, acc.: 97.66%] [G loss: 1.247693]
11 [D loss: 0.188189, acc.: 95.31%] [G loss: 2.213648]
12 [D loss: 0.128924, acc.: 98.44%] [G loss: 2.959294]
13 [D loss: 0.236709, acc.: 92.19%] [G loss: 3.641133]
14 [D loss: 0.267118, acc.: 89.84%] [G loss: 3.953944]
15 [D loss: 0.292798, acc.: 89.84%] [G loss: 4.117201]
16 [D loss: 0.331972, acc.: 85.94%] [G loss: 4.907932]
17 [D loss: 0.623602, acc.: 63.28%] [G loss: 4.810585]
18 [D loss: 0.384210

83 [D loss: 1.166116, acc.: 32.81%] [G loss: 1.678153]
84 [D loss: 0.939635, acc.: 46.88%] [G loss: 1.837800]
85 [D loss: 1.007464, acc.: 39.84%] [G loss: 1.487799]
86 [D loss: 1.076129, acc.: 35.16%] [G loss: 1.406471]
87 [D loss: 1.094180, acc.: 38.28%] [G loss: 1.486233]
88 [D loss: 0.888131, acc.: 45.31%] [G loss: 1.759863]
89 [D loss: 0.888994, acc.: 45.31%] [G loss: 1.663836]
90 [D loss: 1.057400, acc.: 35.16%] [G loss: 1.523679]
91 [D loss: 1.105454, acc.: 33.59%] [G loss: 1.662874]
92 [D loss: 1.044236, acc.: 39.84%] [G loss: 1.909586]
93 [D loss: 0.890415, acc.: 47.66%] [G loss: 2.132244]
94 [D loss: 1.032959, acc.: 39.84%] [G loss: 1.819712]
95 [D loss: 0.984215, acc.: 39.84%] [G loss: 2.089205]
96 [D loss: 0.993498, acc.: 47.66%] [G loss: 2.487646]
97 [D loss: 0.955772, acc.: 46.09%] [G loss: 2.072729]
98 [D loss: 1.308281, acc.: 23.44%] [G loss: 2.217638]
99 [D loss: 1.070543, acc.: 33.59%] [G loss: 2.063104]
100 [D loss: 1.232951, acc.: 26.56%] [G loss: 2.537574]
101 [D lo

165 [D loss: 0.901289, acc.: 43.75%] [G loss: 3.287732]
166 [D loss: 0.928115, acc.: 41.41%] [G loss: 2.589167]
167 [D loss: 1.140133, acc.: 28.12%] [G loss: 2.598400]
168 [D loss: 1.024439, acc.: 39.84%] [G loss: 2.830640]
169 [D loss: 1.069847, acc.: 35.94%] [G loss: 2.523013]
170 [D loss: 0.992087, acc.: 39.84%] [G loss: 2.832453]
171 [D loss: 1.298416, acc.: 26.56%] [G loss: 3.026645]
172 [D loss: 0.992312, acc.: 35.94%] [G loss: 2.645119]
173 [D loss: 1.172110, acc.: 38.28%] [G loss: 2.400026]
174 [D loss: 1.088286, acc.: 37.50%] [G loss: 2.111156]
175 [D loss: 1.045326, acc.: 38.28%] [G loss: 2.614990]
176 [D loss: 0.889842, acc.: 50.00%] [G loss: 2.564324]
177 [D loss: 0.931976, acc.: 46.09%] [G loss: 2.697265]
178 [D loss: 1.045418, acc.: 38.28%] [G loss: 2.405587]
179 [D loss: 1.065800, acc.: 34.38%] [G loss: 2.647046]
180 [D loss: 0.888666, acc.: 48.44%] [G loss: 3.085564]
181 [D loss: 0.846230, acc.: 53.12%] [G loss: 2.703653]
182 [D loss: 1.042701, acc.: 43.75%] [G loss: 2.

247 [D loss: 1.058047, acc.: 35.94%] [G loss: 2.521127]
248 [D loss: 0.769626, acc.: 57.03%] [G loss: 2.292058]
249 [D loss: 0.946840, acc.: 46.88%] [G loss: 2.843048]
250 [D loss: 0.695114, acc.: 63.28%] [G loss: 3.069894]
251 [D loss: 0.660063, acc.: 63.28%] [G loss: 2.678513]
252 [D loss: 0.626198, acc.: 62.50%] [G loss: 2.557710]
253 [D loss: 0.639102, acc.: 67.97%] [G loss: 2.191968]
254 [D loss: 0.596745, acc.: 67.19%] [G loss: 1.921989]
255 [D loss: 0.544408, acc.: 72.66%] [G loss: 2.210475]
256 [D loss: 0.829616, acc.: 57.81%] [G loss: 2.353565]
257 [D loss: 0.545818, acc.: 71.88%] [G loss: 1.625230]
258 [D loss: 0.794398, acc.: 58.59%] [G loss: 2.518089]
259 [D loss: 0.723610, acc.: 60.94%] [G loss: 2.926733]
260 [D loss: 0.912323, acc.: 48.44%] [G loss: 3.192193]
261 [D loss: 0.733034, acc.: 59.38%] [G loss: 3.182277]
262 [D loss: 1.010888, acc.: 43.75%] [G loss: 2.919740]
263 [D loss: 0.779599, acc.: 55.47%] [G loss: 2.829484]
264 [D loss: 0.989332, acc.: 42.97%] [G loss: 3.

...

4593 [D loss: 0.017793, acc.: 100.00%] [G loss: 5.917614]
4594 [D loss: 0.023482, acc.: 100.00%] [G loss: 4.651110]
4595 [D loss: 0.025006, acc.: 100.00%] [G loss: 5.263299]
4596 [D loss: 0.017756, acc.: 100.00%] [G loss: 5.825601]
4597 [D loss: 0.014509, acc.: 100.00%] [G loss: 5.090351]
4598 [D loss: 0.060324, acc.: 98.44%] [G loss: 4.824814]
4599 [D loss: 0.039047, acc.: 99.22%] [G loss: 4.979934]
4600 [D loss: 0.037207, acc.: 99.22%] [G loss: 4.224666]
4601 [D loss: 0.068417, acc.: 98.44%] [G loss: 4.292796]
4602 [D loss: 0.037964, acc.: 100.00%] [G loss: 5.287251]
4603 [D loss: 0.143861, acc.: 96.88%] [G loss: 6.023922]
4604 [D loss: 0.026953, acc.: 99.22%] [G loss: 6.712440]
4605 [D loss: 0.024334, acc.: 100.00%] [G loss: 5.651324]
4606 [D loss: 0.015668, acc.: 100.00%] [G loss: 5.105810]
4607 [D loss: 0.032553, acc.: 99.22%] [G loss: 5.530177]
4608 [D loss: 0.032219, acc.: 100.00%] [G loss: 5.796839]
4609 [D loss: 0.041330, acc.: 99.22%] [G loss: 6.152045]
4610 [D loss: 0.031191

4675 [D loss: 0.029470, acc.: 100.00%] [G loss: 6.289012]
4676 [D loss: 0.090400, acc.: 96.88%] [G loss: 5.355861]
4677 [D loss: 0.092086, acc.: 97.66%] [G loss: 6.148308]
4678 [D loss: 0.037520, acc.: 99.22%] [G loss: 6.092243]
4679 [D loss: 0.021771, acc.: 100.00%] [G loss: 6.363899]
4680 [D loss: 0.015268, acc.: 100.00%] [G loss: 5.684275]
4681 [D loss: 0.060319, acc.: 97.66%] [G loss: 5.026098]
4682 [D loss: 0.066199, acc.: 98.44%] [G loss: 4.884508]
4683 [D loss: 0.014743, acc.: 100.00%] [G loss: 4.719531]
4684 [D loss: 0.029858, acc.: 100.00%] [G loss: 4.458304]
4685 [D loss: 0.056711, acc.: 99.22%] [G loss: 4.938863]
4686 [D loss: 0.058725, acc.: 100.00%] [G loss: 4.126840]
4687 [D loss: 0.056332, acc.: 98.44%] [G loss: 5.466325]
4688 [D loss: 0.046142, acc.: 98.44%] [G loss: 4.212867]
4689 [D loss: 0.022657, acc.: 100.00%] [G loss: 4.125762]
4690 [D loss: 0.048753, acc.: 98.44%] [G loss: 5.256689]
4691 [D loss: 0.101053, acc.: 96.09%] [G loss: 4.529869]
4692 [D loss: 0.022227, 

4757 [D loss: 0.118084, acc.: 96.88%] [G loss: 4.722428]
4758 [D loss: 0.041576, acc.: 98.44%] [G loss: 6.131407]
4759 [D loss: 0.021141, acc.: 100.00%] [G loss: 6.546240]
4760 [D loss: 0.031735, acc.: 100.00%] [G loss: 5.162732]
4761 [D loss: 0.033987, acc.: 99.22%] [G loss: 5.008973]
4762 [D loss: 0.056189, acc.: 99.22%] [G loss: 3.706687]
4763 [D loss: 0.062688, acc.: 99.22%] [G loss: 4.694191]
4764 [D loss: 0.015000, acc.: 100.00%] [G loss: 5.566322]
4765 [D loss: 0.102121, acc.: 96.09%] [G loss: 4.442095]
4766 [D loss: 0.066811, acc.: 100.00%] [G loss: 6.589728]
4767 [D loss: 0.029491, acc.: 100.00%] [G loss: 6.363060]
4768 [D loss: 0.054850, acc.: 100.00%] [G loss: 5.122527]
4769 [D loss: 0.061113, acc.: 99.22%] [G loss: 4.727262]
4770 [D loss: 0.018579, acc.: 100.00%] [G loss: 5.245220]
4771 [D loss: 0.022969, acc.: 100.00%] [G loss: 5.424835]
4772 [D loss: 0.016369, acc.: 100.00%] [G loss: 4.800055]
4773 [D loss: 0.026929, acc.: 100.00%] [G loss: 5.975128]
4774 [D loss: 0.09871

4839 [D loss: 0.003453, acc.: 100.00%] [G loss: 13.041672]
4840 [D loss: 0.005819, acc.: 100.00%] [G loss: 8.824099]
4841 [D loss: 0.046036, acc.: 98.44%] [G loss: 7.477077]
4842 [D loss: 0.007960, acc.: 100.00%] [G loss: 7.112982]
4843 [D loss: 0.035189, acc.: 99.22%] [G loss: 7.017035]
4844 [D loss: 0.006479, acc.: 100.00%] [G loss: 7.334867]
4845 [D loss: 0.023607, acc.: 100.00%] [G loss: 6.553970]
4846 [D loss: 0.033475, acc.: 99.22%] [G loss: 6.334785]
4847 [D loss: 0.037570, acc.: 98.44%] [G loss: 6.591571]
4848 [D loss: 0.012099, acc.: 100.00%] [G loss: 7.153947]
4849 [D loss: 0.011124, acc.: 100.00%] [G loss: 5.746155]
4850 [D loss: 0.023189, acc.: 100.00%] [G loss: 6.139487]
4851 [D loss: 0.020821, acc.: 99.22%] [G loss: 4.918527]
4852 [D loss: 0.039674, acc.: 97.66%] [G loss: 7.166524]
4853 [D loss: 0.013430, acc.: 100.00%] [G loss: 7.813524]
4854 [D loss: 0.008684, acc.: 100.00%] [G loss: 7.302725]
4855 [D loss: 0.009379, acc.: 100.00%] [G loss: 6.315958]
4856 [D loss: 0.008

4921 [D loss: 0.006140, acc.: 100.00%] [G loss: 6.851224]
4922 [D loss: 0.014030, acc.: 100.00%] [G loss: 5.971147]
4923 [D loss: 0.032119, acc.: 99.22%] [G loss: 6.210890]
4924 [D loss: 0.012336, acc.: 100.00%] [G loss: 5.721382]
4925 [D loss: 0.027458, acc.: 99.22%] [G loss: 6.585099]
4926 [D loss: 0.014637, acc.: 100.00%] [G loss: 6.248518]
4927 [D loss: 0.047897, acc.: 99.22%] [G loss: 6.539993]
4928 [D loss: 0.025023, acc.: 100.00%] [G loss: 6.403378]
4929 [D loss: 0.040002, acc.: 99.22%] [G loss: 5.121815]
4930 [D loss: 0.024173, acc.: 100.00%] [G loss: 6.084396]
4931 [D loss: 0.010793, acc.: 100.00%] [G loss: 5.529291]
4932 [D loss: 0.171572, acc.: 91.41%] [G loss: 7.371586]
4933 [D loss: 0.044619, acc.: 98.44%] [G loss: 6.394723]
4934 [D loss: 0.026855, acc.: 99.22%] [G loss: 7.956596]
4935 [D loss: 0.025048, acc.: 99.22%] [G loss: 7.852030]
4936 [D loss: 0.005580, acc.: 100.00%] [G loss: 5.623312]
4937 [D loss: 0.013952, acc.: 100.00%] [G loss: 4.282748]
4938 [D loss: 0.103192

In [0]:
discriminator.save(os.path.join(path,"discriminator.h5"))

In [0]:
generator.save(os.path.join(path,"generator.h5"))

In [0]:
combined_model.save(os.path.join(path,"combined_model.h5"))